In [2]:
import os
import glob
import numpy as np
import pandas as pd
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

from datetime import datetime, timedelta

## Load data function

In [1]:
# def read_csv_file(stock, date):
#     year = datetime.strptime(date, "%Y%m%d").year

#     # Define possible base directories
#     base_dirs = [
#         f"{stock}/{year}/{year}_new/",
#         f"{stock}/{year}/",
#         f"{stock}/{year}_new/"
#     ]

#     filename_pattern = f"{stock}_{date}_*_Intraday_Preprocessed.csv"

#     for base_dir in base_dirs:
#         search_pattern = os.path.join(base_dir, filename_pattern)
#         matching_files = glob.glob(search_pattern)

#         if matching_files:
#             # Load the first matching file
#             return pd.read_csv(matching_files[0])

#     print("File not found in any of the expected locations:")
#     for base_dir in base_dirs:
#         print(f"  {os.path.join(base_dir, filename_pattern)}")

#     return None

In [3]:
def build_yearly_frames(stock):
    """
    Builds 12 expiry-cycle frames from intraday option data files using pandas.

    Filters out rows where:
    - price_problem == True
    - is_tradable == False

    """
    base_dirs = [
        f"{stock}/2024/2024_new/",
        f"{stock}/2024/",
        f"{stock}/2024_new/"
    ]

    file_paths = []
    seen_files = set()

    # Search in all directories
    for base_path in base_dirs:
        pattern = os.path.join(base_path, f"{stock}_*_Intraday_Preprocessed.csv")
        for path in glob.glob(pattern):
            fname = os.path.basename(path)
            if fname not in seen_files:
                file_paths.append(path)
                seen_files.add(fname)

    file_paths.sort()

    # Extract date pairs (start_date, expiry_date)
    date_pairs = []
    for path in file_paths:
        fname = os.path.basename(path)
        try:
            parts = fname.split('_')
            start_date = parts[1]
            expiry_date = parts[2]
            date_pairs.append((start_date, expiry_date, path))
        except IndexError:
            continue

    date_pairs.sort(key=lambda x: x[0])

    frames = []
    used_indices = set()
    current_index = 0

    while len(frames) < 12 and current_index < len(date_pairs):
        frame_paths = []
        start_date, expiry_date, _ = date_pairs[current_index]
        frame_start = start_date
        frame_expiry = expiry_date

        # Collect paths for current expiry cycle
        for i in range(current_index, len(date_pairs)):
            s_date, e_date, path = date_pairs[i]
            if s_date >= frame_start and s_date <= frame_expiry:
                frame_paths.append(path)
                used_indices.add(i)

        if frame_paths:
            dfs = []
            for path in frame_paths:
                df = pd.read_csv(path)

                # Filter: keep only rows where price_problem == False and is_tradable == True
                if "price_problem" in df.columns and "is_tradable" in df.columns:
                    df = df[(df["price_problem"] == False) & (df["is_tradable"] == True)]

                dfs.append(df)

            if dfs:
                frame_df = pd.concat(dfs, axis=0, ignore_index=True)
                frames.append(frame_df)

        # Move to next frame: after expiry
        next_start_date = (datetime.strptime(frame_expiry, "%Y%m%d") + timedelta(days=1)).strftime("%Y%m%d")
        found_next = False
        for i in range(current_index + 1, len(date_pairs)):
            if i in used_indices:
                continue
            if date_pairs[i][0] >= next_start_date:
                current_index = i
                found_next = True
                break

        if not found_next:
            break

    return frames

In [9]:
def process_expiry_frame(frame_df, cash, lot_s, allocated_lots, hedge_interval_minutes):
    """
    Short an ATM straddle at 9:20 AM on the first day of the expiry cycle.
    Dynamically hedge delta exposure using futures based on Spot price.

    Args:
        frame_df (pd.DataFrame): Data for one expiry cycle (e.g., s_1[i]).
        cash (float): Starting capital.
        lot_s (int): Lot size for the instrument.
        allocated_lots (int): Total quantity to allocate.
        hedge_interval_minutes (int): Interval in minutes for iteration.
    """

    if frame_df.empty:
        print("Empty frame — skipping.")
        return None

    capital = cash
    depth = 2
    stock_name = frame_df["Instrument"][0]
    portfolio_log = {}
    net_futures_held = 0 # Track current futures position
    value_held_for_futures = 0 # Track value of futures held


    frame_df['Date Time'] = pd.to_datetime(frame_df['Date Time'])
    frame_df['date'] = frame_df['Date Time'].dt.date

    # Get first date of expiry cycle
    first_date = frame_df['date'].min()
    first_day_df = frame_df[frame_df['date'] == first_date]
    target_time = datetime.strptime("09:20:00", "%H:%M:%S").time()
    # Get the earliest datetime of that day
    first_datetime = first_day_df['Date Time'].min()

    # Take snapshot at 9:20 AM on the first day
    snapshot_920 = first_day_df[first_day_df['Date Time'].dt.time == target_time]

    if snapshot_920.empty:
        print("No data at 9:20 AM on the first day — skipping straddle.")
        return None

    # Identify ATM strike based on Spot
    spot_price = snapshot_920['Spot'].iloc[0]
    available_strikes = sorted(snapshot_920['Strike'].unique(), key=lambda x: abs(x - spot_price))
    
    atm_strike = None
    depth_copy = depth
    # Find closest strike with both CE and PE data
    for strike in available_strikes:
        has_ce = not snapshot_920[(snapshot_920['Strike'] == strike) & (snapshot_920['Type'] == 'CE')].empty
        has_pe = not snapshot_920[(snapshot_920['Strike'] == strike) & (snapshot_920['Type'] == 'PE')].empty
        if has_ce and has_pe:
            atm_strike = strike
            break
        else :
            depth_copy -= 1
            if depth_copy == 0:
                print("Error in getting ATM straddle")
                return None

    if atm_strike is None:
        print("No common strike with both CE and PE found at 9:20 AM — skipping straddle.")
        return None

    # Pull CE and PE rows for ATM strike
    atm_ce = snapshot_920[(snapshot_920['Strike'] == atm_strike) & (snapshot_920['Type'] == 'CE')]
    atm_pe = snapshot_920[(snapshot_920['Strike'] == atm_strike) & (snapshot_920['Type'] == 'PE')]

    ce_bid = atm_ce['BidPrice'].iloc[0]
    pe_bid = atm_pe['BidPrice'].iloc[0]

    # Calculate number of lots to short
    num_lots = int(allocated_lots // lot_s)
    if num_lots == 0:
        print("Allocated quantity too small for even one lot.")
        return None

    # Add premium received
    premium_received = (ce_bid + pe_bid) * lot_s * num_lots

    # Construct keys
    ce_key = f"{stock_name}_{atm_strike}_CE"
    pe_key = f"{stock_name}_{atm_strike}_PE"

    # Create the portfolio entry
    portfolio = {
        ce_key: -lot_s * num_lots,  # short CE
        pe_key: -lot_s * num_lots,  # short PE
        "premium_received_from_straddle": premium_received,
        "pnl": premium_received  # Initial PnL is just the premium received
    }

    # Store in a log dictionary keyed by first_datetime
    portfolio_log = {}
    portfolio_log[first_datetime] = portfolio
    
    # Generate all hedge timestamps up to 15:30 on each day (excluding 15:30 on last day)
    def generate_hedge_intervals(df, interval_minutes):
        unique_dates = sorted(df['date'].unique())
        hedge_times = []

        start_time = datetime.strptime("09:20:00", "%H:%M:%S").time()
        interval = timedelta(minutes=interval_minutes)
        first_hedge_time = (datetime.combine(datetime.today(), start_time) + interval).time()
        end_time = datetime.strptime("15:30:00", "%H:%M:%S").time()

        for i, date in enumerate(unique_dates):
            # Adjusted start time: 9:20 AM + interval
            current = datetime.combine(date, first_hedge_time)
            day_end = datetime.combine(date, end_time)

            # Skip 15:30 on the last day
            if i == len(unique_dates) - 1:
                day_end -= interval

            while current <= day_end:
                hedge_times.append(current)
                current += interval

        return hedge_times

    hedge_timestamps = generate_hedge_intervals(frame_df, hedge_interval_minutes)

    delta_log = []
    
    # Start hedging loop
    for t in hedge_timestamps:
        snapshot = frame_df[frame_df['Date Time'] == t]
        available_strikes = sorted(snapshot['Strike'].unique(), key=lambda x: abs(x - atm_strike))
        depth_copy = depth
        for strike in available_strikes:
            has_ce = not snapshot[(snapshot['Strike'] == strike) & (snapshot['Type'] == 'CE')].empty
            has_pe = not snapshot[(snapshot['Strike'] == strike) & (snapshot['Type'] == 'PE')].empty
            if has_ce and has_pe:
                required_strike = strike
                break
            else:
                depth_copy -= 1
                if depth == 0:
                    print(f"Skipping timestamp {t} — No valid ATM straddle found.")
                    break
        else:
            # If we found a valid strike, continue
            continue

        ce_row = snapshot[(snapshot['Strike'] == required_strike) & (snapshot['Type'] == 'CE')]
        pe_row = snapshot[(snapshot['Strike'] == required_strike) & (snapshot['Type'] == 'PE')]

        delta_ce = ce_row['Delta'].iloc[0]
        delta_pe = pe_row['Delta'].iloc[0]
        initial_delta = (delta_ce + delta_pe) * lot_s * num_lots + net_futures_held

        if initial_delta != 0:
            if abs(initial_delta) // lot_s >= 1:
                # Round adjustment to nearest lot
                hedge_lots = int(abs(initial_delta) // lot_s) * lot_s
                action = 'short' if initial_delta > 0 else 'long'
                futures_price = snapshot['Spot'].iloc[0]
                price_to_hedge = futures_price * hedge_lots

                # Update net futures held
                if action == 'long':
                    value_held_for_futures -= price_to_hedge 
                    net_futures_held += hedge_lots
                else:
                    value_held_for_futures += price_to_hedge
                    net_futures_held -= hedge_lots
            else:
                # Delta too small — no hedge
                pass
        
        delta_log.append({
            'time': t,
            'delta_before_hedging_at_this_time': initial_delta,
            'delta_after_hedging_at_this_time':(delta_ce + delta_pe) * lot_s * num_lots + net_futures_held,
            'futures_held': net_futures_held,
            'need_hedge': abs(initial_delta) > lot_s
        })

        vwap_of_futures_held = value_held_for_futures / abs(net_futures_held) if net_futures_held != 0 else 0
        portfolio = {
            ce_key: -lot_s * num_lots,  # short CE
            pe_key: -lot_s * num_lots,  # short PE
            "vwap_of_futures_held": vwap_of_futures_held,
            "net_futures_held": net_futures_held,
            "pnl": value_held_for_futures + premium_received
        }
        portfolio_log[t] = portfolio

    # After the hedge loop finishes
    last_date = sorted(frame_df['date'].unique())[-1]
    unwind_time = datetime.combine(last_date, datetime.strptime("15:30:00", "%H:%M:%S").time())

    # Snapshot at 15:30 on last day
    snapshot_1530 = frame_df[frame_df['Date Time'] == unwind_time]

    # if snapshot_1530.empty:
    #     print("No data at 15:30 on last day — cannot unwind positions properly.")
    #     return None
    # else:
    #     ce_ask = snapshot_1530[(snapshot_1530['Strike'] == atm_strike) & (snapshot_1530['Type'] == 'CE')]['AskPrice']
    #     pe_ask = snapshot_1530[(snapshot_1530['Strike'] == atm_strike) & (snapshot_1530['Type'] == 'PE')]['AskPrice']

    #     if ce_ask.empty or pe_ask.empty:
    #         print("Missing CE or PE ask price at 15:30 on last day — skipping unwind of options.")
    #     else:
    #         ce_ask_price = ce_ask.iloc[0]
    #         pe_ask_price = pe_ask.iloc[0]

    # Close any remaining futures hedge
    if net_futures_held != 0:
        futures_price = snapshot_1530['Spot'].iloc[0]
        final_cost_for_futures = futures_price * abs(net_futures_held)

        if net_futures_held > 0:
            # Sell futures to close long
            action = 'sell_to_close_futures'
            portfolio = {
                "pnl": value_held_for_futures + premium_received + final_cost_for_futures - abs(atm_strike - snapshot_1530['Spot'].iloc[0]) * lot_s * num_lots
            }
        else:
            # Buy futures to close short
            action = 'buy_to_close_futures'
            portfolio = {
                "pnl": value_held_for_futures + premium_received - final_cost_for_futures - abs(atm_strike - snapshot_1530['Spot'].iloc[0]) * lot_s * num_lots
            }

        net_futures_held = 0  # Reset position
        portfolio_log[unwind_time] = portfolio
    else:
        action = 'no_futures_to_close'
        portfolio_log[unwind_time] = {
            "pnl": value_held_for_futures + premium_received - abs(atm_strike - snapshot_1530['Spot'].iloc[0]) * lot_s * num_lots
        }
    return portfolio_log, delta_log, action, snapshot_1530, atm_strike, net_futures_held, value_held_for_futures

In [5]:
stocks = ["ASIANPAINT", "BAJAJ-AUTO", "BAJFINANCE", "HDFCBANK", "ICICIBANK", "RELIANCE", "SBIN", "TATAMOTORS", "TCS", "TITAN"] # Available "ASIANPAINT", "BAJAJ-AUTO", "BAJFINANCE", "HDFCBANK", "ICICIBANK", "RELIANCE", "SBIN", "TATAMOTORS", "TCS", "TITAN"
weight = [1/len(stocks)] * len(stocks) # weights for the assets

# Lot_sizes
df = pd.read_csv('NIFTY_200_Lot_Size.csv')
df['Symbol'] = df['Symbol'].str.strip().str.upper()
df['LOT_SIZE'] = pd.to_numeric(df['LOT_SIZE'], errors='coerce')

lot_sizes = []

for symbol in stocks:
    symbol = symbol.strip().upper()
    lot_size_row = df[df['Symbol'] == symbol]

    if not lot_size_row.empty:
        lot_size = lot_size_row['LOT_SIZE'].values[0]
    else:
        lot_size = None

    lot_sizes.append(lot_size)

In [6]:
stock_frames = []
result = []

In [7]:
stock_frames.append(build_yearly_frames(stocks[0]))

In [8]:
p_log, d_log = process_expiry_frame(stock_frames[0][0], cash=0, lot_s=lot_sizes[0], allocated_lots=1000, hedge_interval_minutes=5)

Missing CE or PE ask price at 15:30 on last day — skipping unwind of options.


TypeError: cannot unpack non-iterable NoneType object

In [57]:
p_log

{Timestamp('2024-01-01 09:16:00'): {'BAJAJ-AUTO_6800.0_CE': -975.0,
  'BAJAJ-AUTO_6800.0_PE': -975.0,
  'premium_received_from_straddle': 400140.0},
 datetime.datetime(2024, 1, 1, 9, 25): {'BAJAJ-AUTO_6800.0_CE': -975.0,
  'BAJAJ-AUTO_6800.0_PE': -975.0,
  'vwap_of_futures_held': 0,
  'net_futures_held': 0,
  'pnl': 0},
 datetime.datetime(2024, 1, 1, 9, 30): {'BAJAJ-AUTO_6800.0_CE': -975.0,
  'BAJAJ-AUTO_6800.0_PE': -975.0,
  'vwap_of_futures_held': 0,
  'net_futures_held': 0,
  'pnl': 0},
 datetime.datetime(2024, 1, 1, 9, 35): {'BAJAJ-AUTO_6800.0_CE': -975.0,
  'BAJAJ-AUTO_6800.0_PE': -975.0,
  'vwap_of_futures_held': 0,
  'net_futures_held': 0,
  'pnl': 0},
 datetime.datetime(2024, 1, 1, 9, 40): {'BAJAJ-AUTO_6800.0_CE': -975.0,
  'BAJAJ-AUTO_6800.0_PE': -975.0,
  'vwap_of_futures_held': 0,
  'net_futures_held': 0,
  'pnl': 0},
 datetime.datetime(2024, 1, 1, 9, 45): {'BAJAJ-AUTO_6800.0_CE': -975.0,
  'BAJAJ-AUTO_6800.0_PE': -975.0,
  'vwap_of_futures_held': 0,
  'net_futures_held': 0,

In [58]:
# Number of last elements to get
n = 3

# Get last n items as a list of tuples (key, value)
last_items = list(p_log.items())[-n:]

# Optionally convert back to a dict
last_dict = dict(last_items)

print(last_dict)

{datetime.datetime(2024, 1, 25, 15, 15): {'BAJAJ-AUTO_6800.0_CE': -975.0, 'BAJAJ-AUTO_6800.0_PE': -975.0, 'vwap_of_futures_held': 7178.1675, 'net_futures_held': -900.0, 'pnl': 6460350.75}, datetime.datetime(2024, 1, 25, 15, 20): {'BAJAJ-AUTO_6800.0_CE': -975.0, 'BAJAJ-AUTO_6800.0_PE': -975.0, 'vwap_of_futures_held': 7178.1675, 'net_futures_held': -900.0, 'pnl': 6460350.75}, datetime.datetime(2024, 1, 25, 15, 25): {'BAJAJ-AUTO_6800.0_CE': -975.0, 'BAJAJ-AUTO_6800.0_PE': -975.0, 'vwap_of_futures_held': 7178.1675, 'net_futures_held': -900.0, 'pnl': 6460350.75}}


In [59]:
d_log

[{'time': datetime.datetime(2024, 1, 1, 9, 25),
  'delta_before_hedging_at_this_time': 2.1399179425785064,
  'delta_after_hedging_at_this_time': 2.1399179425785064,
  'futures_held': 0,
  'need_hedge': False},
 {'time': datetime.datetime(2024, 1, 1, 9, 30),
  'delta_before_hedging_at_this_time': 25.265322836638457,
  'delta_after_hedging_at_this_time': 25.265322836638457,
  'futures_held': 0,
  'need_hedge': False},
 {'time': datetime.datetime(2024, 1, 1, 9, 35),
  'delta_before_hedging_at_this_time': 42.03638525428488,
  'delta_after_hedging_at_this_time': 42.03638525428488,
  'futures_held': 0,
  'need_hedge': False},
 {'time': datetime.datetime(2024, 1, 1, 9, 40),
  'delta_before_hedging_at_this_time': 29.223422690698442,
  'delta_after_hedging_at_this_time': 29.223422690698442,
  'futures_held': 0,
  'need_hedge': False},
 {'time': datetime.datetime(2024, 1, 1, 9, 45),
  'delta_before_hedging_at_this_time': 28.392238363785793,
  'delta_after_hedging_at_this_time': 28.39223836378579

In [63]:
d_log[-3:]

[{'time': datetime.datetime(2024, 1, 25, 15, 15),
  'delta_before_hedging_at_this_time': -51.03582885620881,
  'delta_after_hedging_at_this_time': -51.03582885620881,
  'futures_held': -900.0,
  'need_hedge': False},
 {'time': datetime.datetime(2024, 1, 25, 15, 20),
  'delta_before_hedging_at_this_time': 66.88328518183675,
  'delta_after_hedging_at_this_time': 66.88328518183675,
  'futures_held': -900.0,
  'need_hedge': False},
 {'time': datetime.datetime(2024, 1, 25, 15, 25),
  'delta_before_hedging_at_this_time': 12.505608557268033,
  'delta_after_hedging_at_this_time': 12.505608557268033,
  'futures_held': -900.0,
  'need_hedge': False}]